# Upload Structure

In [1]:
from __future__ import print_function

from aiida import load_dbenv, is_dbenv_loaded
from aiida.backends import settings
if not is_dbenv_loaded():
    load_dbenv(profile=settings.AIIDADB_PROFILE)

from aiida.orm.querybuilder import QueryBuilder
from aiida.orm.data.structure import StructureData
from aiida.orm.calculation import Calculation
from aiida.orm.data.cif import CifData

import ase.io
import sys
import numpy as np
import ipywidgets as ipw
from base64 import b64decode
from IPython.display import display, clear_output, Image
from fileupload import FileUploadWidget
from tempfile import NamedTemporaryFile
from base64 import b64encode


import nglview

In [2]:
PREPROCESS_VERSION = 0.04
iteration = 0
def preprocess_newbies():
    global struct, iteration
    qb = QueryBuilder()
    qb.append(CifData, filters={
        'or':[
                {'extras': {'!has_key': 'preprocess_version'}},
                {'extras.preprocess_version': {'<': PREPROCESS_VERSION}},
            ],
     })
#    qb.append(CifData, filters={'id':{'==':100}})
    
    qb.limit(100)
    #i=0
    #lst = qb.all()
    #print ("size: {}".format(sys.getsizeof(lst)))
    for m in qb.all(batch_size=100): # iterall() would interfere with set_extra()
        n = m[0]
        try:
            #print("preprocess started")
            preprocess_one(n)
            n.set_extra('preprocess_successful', True)
            n.set_extra('preprocess_version', PREPROCESS_VERSION)
            #print("Preprocessed PK %d"%n.pk)
        except Exception as e:
            n.set_extra('preprocess_successful', False)
            n.set_extra('preprocess_error', str(e))
            n.set_extra('preprocess_version', PREPROCESS_VERSION)
            print("Failed to preprocess PK %d: %s"%(n.pk, e))
        iteration+=1
    print (iteration)
    

In [5]:
def preprocess_one(structure):
    ase_struct = structure.get_ase()
    formula = ase_struct.get_chemical_formula()
    structure.set_extra('formula', '{}'.format(formula))
    structure.set_extra('structure_description', structure.description)
    
    # output parameters
    params_dict = structure.get_outputs()[0].get_outputs()[0].get_dict()
    #print (params_dict)
    #print("Start extras")
    #sys.stdout.flush()
    for key, value in params_dict.iteritems():
        try:
            structure.set_extra(key, float(value))
        except:
            structure.set_extra(key, value)
    #structure.set_extras(params_dict)
    #print ("Got extras")
    #sys.stdout.flush()
    thumbnail = render_thumbnail(ase_struct)
    structure.set_extra('thumbnail', thumbnail)

In [6]:
def render_thumbnail(atoms):
    tmp = NamedTemporaryFile()
    ase.io.write(tmp.name, atoms, format='png') # does not accept StringIO
    raw = open(tmp.name).read()
    tmp.close()
    return b64encode(raw)

In [ ]:
for i in range(700):
    preprocess_newbies()

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
